In [1]:
import pandas as pd
import re

In [2]:
cat62 = pd.read_csv("data/cat-62.csv")

In [3]:
cat62

,flightid,lat,lon,flightlevel,speed,dt_radar
0,62e589aaa96762725b092b898d48a35e,-0.437116,-0.845674,210.0,266.0,1.654056e+12
1,62e589aaa96762725b092b898d48a35e,-0.410386,-0.821362,105.0,266.0,1.654057e+12
2,62e589aaa96762725b092b898d48a35e,-0.410423,-0.821448,106.0,266.0,1.654057e+12
3,62e589aaa96762725b092b898d48a35e,-0.510224,-0.883870,177.0,221.0,1.654052e+12
4,62e589aaa96762725b092b898d48a35e,-0.490834,-0.873397,210.0,270.0,1.654053e+12
...,...,...,...,...,...,...
37122613,Data final não disponível,NaN,NaN,NaN,NaN,NaN
37122614,Data final não disponível,NaN,NaN,NaN,NaN,NaN
37122615,Data final não disponível,NaN,NaN,NaN,NaN,NaN
37122616,Data final não disponível,NaN,NaN,NaN,NaN,NaN


In [4]:

cat62 = cat62[cat62["flightid"] != "Data final não disponível"]

In [5]:


cat62['dt_radar'] = pd.to_datetime(cat62['dt_radar'], unit='ms')

/tmp/ipykernel_144825/1306218513.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat62['dt_radar'] = pd.to_datetime(cat62['dt_radar'], unit='ms')


In [6]:
bimtra = pd.read_csv("data/bimtra.csv")

In [7]:

bimtra['dt_dep'] = pd.to_datetime(bimtra['dt_dep'], unit='ms')
bimtra['dt_arr'] = pd.to_datetime(bimtra['dt_arr'], unit='ms')

In [8]:
bimtra["target"] = (bimtra["dt_arr"] - bimtra["dt_dep"]) // pd.Timedelta('1s')

In [9]:
bimtra["linha"] = bimtra["origem"] + bimtra["destino"]

In [10]:
bimtra = bimtra.drop_duplicates(subset='flightid')

In [11]:
cat62 = cat62.merge(bimtra, on="flightid", how="left")

In [12]:
cat62["linha"] = cat62["origem"] + cat62["destino"]

In [13]:
resumo_voo = cat62.groupby('flightid').agg({'dt_radar': lambda x: (x.max() - x.min()).total_seconds(),
                                         'lat': ['mean', 'min', 'max', 'std'],
                                         'lon': ['mean', 'min', 'max', 'std'],
                                         'flightlevel': ['mean', 'min', 'max', 'std'],
                                         'speed': ['mean', 'min', 'max', 'std'],
                                         'linha': 'first'})

print(resumo_voo)

                                 dt_radar       lat                      \
                                 <lambda>      mean       min       max   
flightid                                                                  
0000f76a51b99be3f181ea2b201b34ef   6543.0 -0.306555 -0.391881 -0.225116   
00010bb2c49076d56a9815122fefc8aa   4083.0 -0.382988 -0.412354 -0.342865   
00010ed6d09c78b5a37f4df13f524f56   7740.0 -0.322293 -0.412303 -0.227512   
000143522a56fa245543e477c74fd908  10019.0 -0.221275 -0.413506 -0.138145   
00014fa436546dfdc7a67919e9e1ea47   4924.0 -0.407391 -0.477623 -0.342804   
...                                   ...       ...       ...       ...   
fffeba6342187a9ee0caff2526e4ff0e   4563.0 -0.470063 -0.523588 -0.411403   
ffff413d10cfbb298a9411855e80a0f0   6602.0 -0.465791 -0.523506 -0.399242   
ffff62189abb326d9f18ff1412e5de8e   3782.0 -0.383321 -0.410217 -0.341090   
ffffd0e5ce497de7ddf3d34d19b9ce3a   2581.0 -0.407535 -0.413719 -0.400364   
ffffd4331f8b78c6e726eda4e

In [14]:
# Aplicar a função join às colunas do DataFrame
resumo_voo.reset_index(inplace=True)


In [15]:
resumo_voo = resumo_voo.T.reset_index().T

In [16]:
arr = []
for i in range(0, len(resumo_voo.iloc[0].values)):
    arr.append(resumo_voo.iloc[0].values[i] + "_" + resumo_voo.iloc[1].values[i])

In [17]:
resumo_voo = resumo_voo.rename(columns=dict(zip(resumo_voo.columns, arr)))


In [18]:
resumo_voo = resumo_voo.iloc[2:]

In [19]:
resumo = resumo_voo.groupby('linha_first').agg({
    'flightid_': 'count',
    'dt_radar_<lambda>': ['mean', 'min', 'max', 'std'],
    'flightlevel_mean': 'mean',
    'flightlevel_max': 'mean',
    'flightlevel_std': ['std', 'mean'],
    'speed_std': ['mean','std'],
})

# Exibir o resumo
print(resumo)

            flightid_ dt_radar_<lambda>                                \
                count              mean     min      max          std   
linha_first                                                             
SBBRSBCF         2644       3210.663767     0.0   8277.0   506.981575   
SBBRSBCT          959       5524.459854     4.0   8166.0   602.814715   
SBBRSBFL          315       6614.263492     4.0  14041.0   824.447364   
SBBRSBGL          639       4499.599374   362.0   7138.0   539.795848   
SBBRSBGR         3124       4384.066261   238.0  15064.0   636.873657   
...               ...               ...     ...      ...          ...   
SBSVSBKP         1379       7368.569978   121.0  11225.0   761.985305   
SBSVSBPA           79      11771.683544  1683.0  13623.0  1455.223285   
SBSVSBRF         1491        3784.31053     0.0   9182.0   526.762331   
SBSVSBRJ         1630       6706.908589   843.0  10924.0   630.301570   
SBSVSBSP         3519       7789.326229   305.0  15

In [20]:
resumo.reset_index(inplace=True)
resumo.to_csv("data/resumo_voo.csv", index=False)

In [21]:
resumo = pd.read_csv("data/resumo_voo.csv")
resumo

,linha_first,flightid_,dt_radar_<lambda>,dt_radar_<lambda>.1,dt_radar_<lambda>.2,dt_radar_<lambda>.3,flightlevel_mean,flightlevel_max,flightlevel_std,flightlevel_std.1,speed_std,speed_std.1
0,NaN,count,mean,min,max,std,mean,mean,std,mean,mean,std
1,SBBRSBCF,2644,3210.6637670196674,0.0,8277.0,506.9815754584058,223.23009635516672,352.7874432677761,15.414484379289918,113.11099040127084,96.26012004571065,16.617049986812884
2,SBBRSBCT,959,5524.459854014599,4.0,8166.0,602.8147149739482,281.071385902067,367.9061522419187,13.184672048521252,113.58039360634044,84.08358708763087,13.50918093115723
3,SBBRSBFL,315,6614.263492063492,4.0,14041.0,824.4473638920381,291.94466776671055,371.6634920634921,15.232888859932348,115.89589711595086,79.03449466219764,14.21473759918302
4,SBBRSBGL,639,4499.599374021909,362.0,7138.0,539.795847627226,259.66979013256446,367.9154929577465,13.252080132619195,122.03779859280796,92.35050625485451,15.93562582501301
...,...,...,...,...,...,...,...,...,...,...,...,...
133,SBSVSBKP,1379,7368.569978245105,121.0,11225.0,761.98530512265,305.3446206649822,369.22770123277735,17.341259072228155,102.26689170812047,68.52585082945009,16.681647582814314
134,SBSVSBPA,79,11771.683544303798,1683.0,13623.0,1455.2232848701617,324.43509942690906,375.1898734177215,8.228248797344037,99.87921093160097,62.36285594137942,8.362795726130809
135,SBSVSBRF,1491,3784.3105298457413,0.0,9182.0,526.7623314570509,226.64509710011185,362.5083836351442,15.70704196991436,127.34337283196848,97.04344098509465,14.564714473730632
136,SBSVSBRJ,1630,6706.908588957055,843.0,10924.0,630.3015703620225,291.5118729723326,378.08895705521473,12.702016763699223,123.87140594500069,82.84562463100339,13.215119734561949


In [22]:
resumo

,linha_first,flightid_,dt_radar_<lambda>,dt_radar_<lambda>.1,dt_radar_<lambda>.2,dt_radar_<lambda>.3,flightlevel_mean,flightlevel_max,flightlevel_std,flightlevel_std.1,speed_std,speed_std.1
0,NaN,count,mean,min,max,std,mean,mean,std,mean,mean,std
1,SBBRSBCF,2644,3210.6637670196674,0.0,8277.0,506.9815754584058,223.23009635516672,352.7874432677761,15.414484379289918,113.11099040127084,96.26012004571065,16.617049986812884
2,SBBRSBCT,959,5524.459854014599,4.0,8166.0,602.8147149739482,281.071385902067,367.9061522419187,13.184672048521252,113.58039360634044,84.08358708763087,13.50918093115723
3,SBBRSBFL,315,6614.263492063492,4.0,14041.0,824.4473638920381,291.94466776671055,371.6634920634921,15.232888859932348,115.89589711595086,79.03449466219764,14.21473759918302
4,SBBRSBGL,639,4499.599374021909,362.0,7138.0,539.795847627226,259.66979013256446,367.9154929577465,13.252080132619195,122.03779859280796,92.35050625485451,15.93562582501301
...,...,...,...,...,...,...,...,...,...,...,...,...
133,SBSVSBKP,1379,7368.569978245105,121.0,11225.0,761.98530512265,305.3446206649822,369.22770123277735,17.341259072228155,102.26689170812047,68.52585082945009,16.681647582814314
134,SBSVSBPA,79,11771.683544303798,1683.0,13623.0,1455.2232848701617,324.43509942690906,375.1898734177215,8.228248797344037,99.87921093160097,62.36285594137942,8.362795726130809
135,SBSVSBRF,1491,3784.3105298457413,0.0,9182.0,526.7623314570509,226.64509710011185,362.5083836351442,15.70704196991436,127.34337283196848,97.04344098509465,14.564714473730632
136,SBSVSBRJ,1630,6706.908588957055,843.0,10924.0,630.3015703620225,291.5118729723326,378.08895705521473,12.702016763699223,123.87140594500069,82.84562463100339,13.215119734561949


In [23]:
esperas = pd.read_csv("data/esperas.csv")
esperas["hora"] = pd.to_datetime(esperas["hora"], unit='ms')
esperas["diaSemana"] = esperas["hora"].dt.dayofweek

esperas["hora"] = esperas["hora"].dt.hour

In [24]:
esperas_hora = esperas.groupby(["aero", "hora"]).agg({"esperas": ["mean", "sum"]})
esperas_semana = esperas.groupby(["aero", "diaSemana"]).agg({"esperas": ["mean", "sum"]})

In [25]:

esperas_aero = esperas.groupby(["aero"]).agg({"esperas": ["mean", "sum"]})

In [26]:
bimtra

,flightid,origem,destino,dt_dep,dt_arr,target,linha
0,fcb2bf90345705318213ae1307c0f901,SBKP,SBRJ,2022-06-01 00:44:57,2022-06-01 01:26:00,2463,SBKPSBRJ
1,c7c5c10716335b048f86d8c52fcba3f2,SBGR,SBRJ,2022-06-01 00:57:01,2022-06-01 01:32:53,2152,SBGRSBRJ
2,3162de7203a972f071d5a48e8f0f4828,SBBR,SBRJ,2022-06-01 00:26:52,2022-06-01 01:43:45,4613,SBBRSBRJ
3,8fcd243e9b9eebbe62ab145ce04b6ab5,SBKP,SBCF,2022-06-01 00:58:40,2022-06-01 01:41:41,2581,SBKPSBCF
4,504a62621cd231d6ab67e674ce538cd3,SBCF,SBFL,2022-06-01 01:00:38,2022-06-01 02:33:05,5547,SBCFSBFL
...,...,...,...,...,...,...,...
278879,12d84d9dd7450b2cac83cdf041f58d67,SBGR,SBRJ,2023-04-15 21:41:11,2023-04-15 22:16:49,2138,SBGRSBRJ
278880,c1e1fcbfd1d1c67c01da384ad104c0df,SBBR,SBRJ,2023-04-15 21:07:10,2023-04-15 22:26:11,4741,SBBRSBRJ
278882,612af5b2b7cc48161bee87965785282b,SBFL,SBRJ,2023-04-15 21:30:27,2023-04-15 22:31:36,3669,SBFLSBRJ
278884,5efa9c4eb13e214f29a73a82cf7822e4,SBSV,SBRJ,2023-04-15 12:00:13,2023-04-15 14:00:05,7192,SBSVSBRJ


In [27]:
bimtra["diaSemana"] = bimtra["dt_dep"].dt.dayofweek

bimtra["hora"] = bimtra["dt_dep"].dt.hour

In [28]:
bimtra

,flightid,origem,destino,dt_dep,dt_arr,target,linha,diaSemana,hora
0,fcb2bf90345705318213ae1307c0f901,SBKP,SBRJ,2022-06-01 00:44:57,2022-06-01 01:26:00,2463,SBKPSBRJ,2,0
1,c7c5c10716335b048f86d8c52fcba3f2,SBGR,SBRJ,2022-06-01 00:57:01,2022-06-01 01:32:53,2152,SBGRSBRJ,2,0
2,3162de7203a972f071d5a48e8f0f4828,SBBR,SBRJ,2022-06-01 00:26:52,2022-06-01 01:43:45,4613,SBBRSBRJ,2,0
3,8fcd243e9b9eebbe62ab145ce04b6ab5,SBKP,SBCF,2022-06-01 00:58:40,2022-06-01 01:41:41,2581,SBKPSBCF,2,0
4,504a62621cd231d6ab67e674ce538cd3,SBCF,SBFL,2022-06-01 01:00:38,2022-06-01 02:33:05,5547,SBCFSBFL,2,1
...,...,...,...,...,...,...,...,...,...
278879,12d84d9dd7450b2cac83cdf041f58d67,SBGR,SBRJ,2023-04-15 21:41:11,2023-04-15 22:16:49,2138,SBGRSBRJ,5,21
278880,c1e1fcbfd1d1c67c01da384ad104c0df,SBBR,SBRJ,2023-04-15 21:07:10,2023-04-15 22:26:11,4741,SBBRSBRJ,5,21
278882,612af5b2b7cc48161bee87965785282b,SBFL,SBRJ,2023-04-15 21:30:27,2023-04-15 22:31:36,3669,SBFLSBRJ,5,21
278884,5efa9c4eb13e214f29a73a82cf7822e4,SBSV,SBRJ,2023-04-15 12:00:13,2023-04-15 14:00:05,7192,SBSVSBRJ,5,12


In [29]:
esperas_hora.columns = esperas_hora.columns.droplevel(0)

In [30]:
esperas_hora.reset_index(inplace=True)

In [31]:
esperas_semana.columns = esperas_semana.columns.droplevel(0)
esperas_semana.reset_index(inplace=True)

In [32]:
esperas_aero.columns = esperas_aero.columns.droplevel(0)
esperas_aero.reset_index(inplace=True)

In [33]:
esperas_semana

,aero,diaSemana,mean,sum
0,SBBR,0,0.009058,10
1,SBBR,1,0.019022,21
2,SBBR,2,0.034420,38
3,SBBR,3,0.022163,25
4,SBBR,4,0.069149,78
...,...,...,...,...
79,SBSV,2,0.011775,13
80,SBSV,3,0.015071,17
81,SBSV,4,0.006206,7
82,SBSV,5,0.003623,4


In [34]:
bimtra = bimtra.merge(esperas_hora, right_on=["aero", "hora"], left_on=["origem", "hora"] , how="left")
bimtra = bimtra.merge(esperas_semana, right_on=["aero", "diaSemana"], left_on=["origem", "diaSemana"] , how="left")

In [35]:
bimtra = bimtra.merge(esperas_aero, right_on=["aero"], left_on=["origem"] , how="left")

In [36]:
bimtra["origem"].unique()

array(['SBKP', 'SBGR', 'SBBR', 'SBCF', 'SBSP', 'SBRJ', 'SBFL', 'SBCT',
       'SBRF', 'SBPA', 'SBSV', 'SBGL'], dtype=object)

In [37]:
bimtra

,flightid,origem,destino,dt_dep,dt_arr,target,linha,diaSemana,hora,aero_x,mean_x,sum_x,aero_y,mean_y,sum_y,aero,mean,sum
0,fcb2bf90345705318213ae1307c0f901,SBKP,SBRJ,2022-06-01 00:44:57,2022-06-01 01:26:00,2463,SBKPSBRJ,2,0,SBKP,0.694444,225,SBKP,0.221014,244,SBKP,0.191872,1492
1,c7c5c10716335b048f86d8c52fcba3f2,SBGR,SBRJ,2022-06-01 00:57:01,2022-06-01 01:32:53,2152,SBGRSBRJ,2,0,SBGR,0.987654,320,SBGR,0.254529,281,SBGR,0.255401,1986
2,3162de7203a972f071d5a48e8f0f4828,SBBR,SBRJ,2022-06-01 00:26:52,2022-06-01 01:43:45,4613,SBBRSBRJ,2,0,SBBR,0.000000,0,SBBR,0.034420,38,SBBR,0.024434,190
3,8fcd243e9b9eebbe62ab145ce04b6ab5,SBKP,SBCF,2022-06-01 00:58:40,2022-06-01 01:41:41,2581,SBKPSBCF,2,0,SBKP,0.694444,225,SBKP,0.221014,244,SBKP,0.191872,1492
4,504a62621cd231d6ab67e674ce538cd3,SBCF,SBFL,2022-06-01 01:00:38,2022-06-01 02:33:05,5547,SBCFSBFL,2,1,SBCF,0.055556,18,SBCF,0.173007,191,SBCF,0.150463,1170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262411,12d84d9dd7450b2cac83cdf041f58d67,SBGR,SBRJ,2023-04-15 21:41:11,2023-04-15 22:16:49,2138,SBGRSBRJ,5,21,SBGR,0.123457,40,SBGR,0.167572,185,SBGR,0.255401,1986
262412,c1e1fcbfd1d1c67c01da384ad104c0df,SBBR,SBRJ,2023-04-15 21:07:10,2023-04-15 22:26:11,4741,SBBRSBRJ,5,21,SBBR,0.074074,24,SBBR,0.004529,5,SBBR,0.024434,190
262413,612af5b2b7cc48161bee87965785282b,SBFL,SBRJ,2023-04-15 21:30:27,2023-04-15 22:31:36,3669,SBFLSBRJ,5,21,SBFL,0.033951,11,SBFL,0.012681,14,SBFL,0.015947,124
262414,5efa9c4eb13e214f29a73a82cf7822e4,SBSV,SBRJ,2023-04-15 12:00:13,2023-04-15 14:00:05,7192,SBSVSBRJ,5,12,SBSV,0.000000,0,SBSV,0.003623,4,SBSV,0.009259,72


In [46]:
metar = pd.read_csv("data/metar.csv")

In [40]:
metar

,hora,metar,aero
0,1654041600000,METAR SBBR 010000Z 07002KT CAVOK 21/08 Q1018=,SBBR
1,1654045200000,METAR SBBR 010100Z 10002KT CAVOK 20/09 Q1019=,SBBR
2,1654048800000,METAR SBBR 010200Z 00000KT CAVOK 16/09 Q1019=,SBBR
3,1654052400000,METAR SBBR 010300Z 27002KT CAVOK 16/09 Q1019=,SBBR
4,1654056000000,METAR SBBR 010400Z 00000KT CAVOK 14/09 Q1018=,SBBR
...,...,...,...
99617,1681585200000,METAR SBSV 151900Z 08008KT 050V110 9999 FEW030...,SBSV
99618,1681588800000,METAR SBSV 152000Z 09007KT 060V120 CAVOK 29/23...,SBSV
99619,1681592400000,METAR SBSV 152100Z 08007KT CAVOK 28/22 Q1011=,SBSV
99620,1681596000000,METAR SBSV 152200Z 08007KT CAVOK 28/22 Q1011=,SBSV


In [43]:
import pandas as pd
from metar.Metar import Metar

# Função para parsear METAR
def parse_metar(metar):
    try:
        metar_obj = Metar(metar)
        return set([attr for attr in dir(metar_obj) if not callable(getattr(metar_obj, attr)) and not attr.startswith("__")])
    except:
        return set()

# Aplicar parse sobre METAR para cada linha
all_possible_info = set()
for metar in metar['metar']:
    all_possible_info = all_possible_info.union(parse_metar(metar))

print(all_possible_info)


{'_unparsed_groups', '_trend_groups', 'wind_gust', '_now', 'station_id', 'handlers', 'ice_accretion_1hr', 'wind_shift_time', '_day', 'remark_handlers', 'wind_speed', 'windshear', 'correction', 'code', 'press', 'weather', 'ice_accretion_6hr', 'max_temp_6hr', 'snowdepth', '_min', 'wind_dir_from', 'min_temp_6hr', 'precip_3hr', 'wind_dir_peak', 'press_sea_level', 'dewpt', 'decode_completed', 'recent', 'peak_wind_time', '_utcdelta', 'cycle', 'time', 'temp', '_unparsed_remarks', '_year', 'wind_speed_peak', 'max_vis_dir', '_remarks', 'precip_1hr', 'type', 'max_temp_24hr', 'wind_dir_to', 'ice_accretion_3hr', 'runway', 'trend_handlers', 'mod', 'precip_6hr', 'wind_dir', 'min_temp_24hr', '_trend', 'precip_24hr', 'max_vis', '_hour', 'sky', 'vis_dir', 'vis', '_month'}


In [49]:
import pandas as pd
from metar.Metar import Metar

# Função para parsear METAR
def parse_metar(metar):
    try:
        metar_obj = Metar(metar)
        temperature = metar_obj.temp.value()
        dewpoint = metar_obj.dewpt.value()
        wind_speed = metar_obj.wind_speed.value()
        wind_direction = metar_obj.wind_dir.value()
        return pd.Series([temperature, dewpoint, wind_speed, wind_direction])
    except:
        return pd.Series([None, None, None, None])

# Aplicar parse sobre METAR para cada linha
metar[['Temperatura', 'Ponto de Orvalho', 'Velocidade do Vento', 'Direção do Vento']] = metar['metar'].apply(parse_metar)


/tmp/ipykernel_144825/2446187406.py:17: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  metar[['Temperatura', 'Ponto de Orvalho', 'Velocidade do Vento', 'Direção do Vento']] = metar['metar'].apply(parse_metar)


In [50]:
metar

,hora,metar,aero,Temperatura,Ponto de Orvalho,Velocidade do Vento,Direção do Vento
0,1654041600000,METAR SBBR 010000Z 07002KT CAVOK 21/08 Q1018=,SBBR,21.0,8.0,2.0,70.0
1,1654045200000,METAR SBBR 010100Z 10002KT CAVOK 20/09 Q1019=,SBBR,20.0,9.0,2.0,100.0
2,1654048800000,METAR SBBR 010200Z 00000KT CAVOK 16/09 Q1019=,SBBR,16.0,9.0,0.0,0.0
3,1654052400000,METAR SBBR 010300Z 27002KT CAVOK 16/09 Q1019=,SBBR,16.0,9.0,2.0,270.0
4,1654056000000,METAR SBBR 010400Z 00000KT CAVOK 14/09 Q1018=,SBBR,14.0,9.0,0.0,0.0
...,...,...,...,...,...,...,...
99617,1681585200000,METAR SBSV 151900Z 08008KT 050V110 9999 FEW030...,SBSV,29.0,23.0,8.0,80.0
99618,1681588800000,METAR SBSV 152000Z 09007KT 060V120 CAVOK 29/23...,SBSV,29.0,23.0,7.0,90.0
99619,1681592400000,METAR SBSV 152100Z 08007KT CAVOK 28/22 Q1011=,SBSV,28.0,22.0,7.0,80.0
99620,1681596000000,METAR SBSV 152200Z 08007KT CAVOK 28/22 Q1011=,SBSV,28.0,22.0,7.0,80.0


In [42]:
metar

,hora,metar,aero,Temperatura,Ponto de Orvalho,Velocidade do Vento,Direção do Vento
0,1654041600000,METAR SBBR 010000Z 07002KT CAVOK 21/08 Q1018=,SBBR,21.0,8.0,2.0,70.0
1,1654045200000,METAR SBBR 010100Z 10002KT CAVOK 20/09 Q1019=,SBBR,20.0,9.0,2.0,100.0
2,1654048800000,METAR SBBR 010200Z 00000KT CAVOK 16/09 Q1019=,SBBR,16.0,9.0,0.0,0.0
3,1654052400000,METAR SBBR 010300Z 27002KT CAVOK 16/09 Q1019=,SBBR,16.0,9.0,2.0,270.0
4,1654056000000,METAR SBBR 010400Z 00000KT CAVOK 14/09 Q1018=,SBBR,14.0,9.0,0.0,0.0
...,...,...,...,...,...,...,...
99617,1681585200000,METAR SBSV 151900Z 08008KT 050V110 9999 FEW030...,SBSV,29.0,23.0,8.0,80.0
99618,1681588800000,METAR SBSV 152000Z 09007KT 060V120 CAVOK 29/23...,SBSV,29.0,23.0,7.0,90.0
99619,1681592400000,METAR SBSV 152100Z 08007KT CAVOK 28/22 Q1011=,SBSV,28.0,22.0,7.0,80.0
99620,1681596000000,METAR SBSV 152200Z 08007KT CAVOK 28/22 Q1011=,SBSV,28.0,22.0,7.0,80.0


In [79]:
import pandas as pd

# DataFrame inicial com as chaves extraídas da string
incomplete_df = pd.DataFrame({
    'station': ['SBBR'],
    'type': ['routine report (automatic report)'],
    'time': ['Fri Sep  1 00:00:00 2023'],
    'temperature': ['21.0 C'],
    'dew point': ['8.0 C'],
    'wind': ['ENE at 2 knots'],
    'visibility': ['10000 meters'],
    'pressure': ['1018.0 mb'],
    'METAR': ['METAR SBBR 010000Z 07002KT CAVOK 21/08 Q1018=']
})



In [80]:

# Extrair as variáveis separadas por espaço da coluna "metar"
variables = metar['metar'].str.split(' ', expand=True)


In [81]:
variables

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,METAR,SBBR,010000Z,07002KT,CAVOK,21/08,Q1018=,None,None,None,None,None,None,None,None,None,None,None,None
1,METAR,SBBR,010100Z,10002KT,CAVOK,20/09,Q1019=,None,None,None,None,None,None,None,None,None,None,None,None
2,METAR,SBBR,010200Z,00000KT,CAVOK,16/09,Q1019=,None,None,None,None,None,None,None,None,None,None,None,None
3,METAR,SBBR,010300Z,27002KT,CAVOK,16/09,Q1019=,None,None,None,None,None,None,None,None,None,None,None,None
4,METAR,SBBR,010400Z,00000KT,CAVOK,14/09,Q1018=,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99617,METAR,SBSV,151900Z,08008KT,050V110,9999,FEW030,29/23,Q1011=,None,None,None,None,None,None,None,None,None,None
99618,METAR,SBSV,152000Z,09007KT,060V120,CAVOK,29/23,Q1011=,None,None,None,None,None,None,None,None,None,None,None
99619,METAR,SBSV,152100Z,08007KT,CAVOK,28/22,Q1011=,None,None,None,None,None,None,None,None,None,None,None,None
99620,METAR,SBSV,152200Z,08007KT,CAVOK,28/22,Q1011=,None,None,None,None,None,None,None,None,None,None,None,None


In [ ]:

# Atualizar as colunas do DataFrame incompleto
incomplete_df['hora'] = metar_df['hora']
incomplete_df['temperature'] = variables[9]
incomplete_df['dew point'] = variables[10]
incomplete_df['wind'] = variables[11] + ' at ' + variables[12] + ' knots'
incomplete_df['visibility'] = variables[13] + ' meters'
incomplete_df['pressure'] = variables[16]

# Remover as colunas irrelevantes
incomplete_df.drop(['METAR'], axis=1, inplace=True)

# Converter a coluna "time" para o tipo datetime
incomplete_df['time'] = pd.to_datetime(incomplete_df['time'])

# Renomear as colunas para torná-las consistentes com o DataFrame incompleto
incomplete_df.rename(columns={'hora': 'time', 'aero': 'station'}, inplace=True)

# Exibir o DataFrame final
print(incomplete_df)


In [77]:
metar["hora"] = pd.to_datetime(metar["hora"], unit='ms')
metar

,hora,metar,aero
0,2022-06-01 00:00:00,METAR SBBR 010000Z 07002KT CAVOK 21/08 Q1018=,SBBR
1,2022-06-01 01:00:00,METAR SBBR 010100Z 10002KT CAVOK 20/09 Q1019=,SBBR
2,2022-06-01 02:00:00,METAR SBBR 010200Z 00000KT CAVOK 16/09 Q1019=,SBBR
3,2022-06-01 03:00:00,METAR SBBR 010300Z 27002KT CAVOK 16/09 Q1019=,SBBR
4,2022-06-01 04:00:00,METAR SBBR 010400Z 00000KT CAVOK 14/09 Q1018=,SBBR
...,...,...,...
99617,2023-04-15 19:00:00,METAR SBSV 151900Z 08008KT 050V110 9999 FEW030...,SBSV
99618,2023-04-15 20:00:00,METAR SBSV 152000Z 09007KT 060V120 CAVOK 29/23...,SBSV
99619,2023-04-15 21:00:00,METAR SBSV 152100Z 08007KT CAVOK 28/22 Q1011=,SBSV
99620,2023-04-15 22:00:00,METAR SBSV 152200Z 08007KT CAVOK 28/22 Q1011=,SBSV


In [38]:
tcprev = pd.read_csv("data/tc-prev.csv")

In [39]:
tcprev["hora"] = pd.to_datetime(tcprev["hora"], unit='ms')

In [40]:
tcprev["diaSemana"] = tcprev["hora"].dt.dayofweek

tcprev["hora"] = tcprev["hora"].dt.hour
tcprev_hora = tcprev.groupby(["aero", "hora"]).agg({"troca": ["mean", "sum"]})
tcprev_sem = tcprev.groupby(["aero", "diaSemana"]).agg({"troca": ["mean", "sum"]})
tcprev_aero = tcprev.groupby(["aero"]).agg({"troca": ["mean", "sum"]})

In [41]:
tcreal = pd.read_csv("data/tc-real.csv")
tcreal["hora"] = pd.to_datetime(tcreal["hora"], unit='ms')
tcreal["diaSemana"] = tcreal["hora"].dt.dayofweek
tcreal["hora"] = tcreal["hora"].dt.hour

In [42]:

tcreal_hora = tcreal.groupby(["aero", "hora"]).agg({"aero": ["count"]})
tcreal_sem = tcreal.groupby(["aero", "diaSemana"]).agg({"aero": ["count"]})
tcreal_aero = tcreal.groupby(["aero"]).agg({"aero": ["count"]})

In [43]:
tcreal_aero

,aero
,count
aero,
BR,1153
CF,912
CT,1509
FL,6151
GL,2491
GR,814
KP,2060
PA,910


In [44]:
tcprev = pd.read_csv("data/tc-prev.csv")

In [46]:
bimtra["aero"] = bimtra["origem"].str[2:]

In [47]:
len(bimtra)

262416

In [50]:
tcprev_hora.reset_index(inplace=True)

In [53]:
tcprev_hora.columns = tcprev_hora.columns.droplevel(0)

In [56]:
tcprev_hora.columns = ["aero", "hora", "troca_prev_mean", "troca_prev_sum"]
tcprev_aero.reset_index(inplace=True)
tcprev_aero.columns = tcprev_aero.columns.droplevel(0)
tcprev_aero.columns = ["aero", "troca_prev_mean", "troca_prev_sum"]
tcprev_sem.reset_index(inplace=True)
tcprev_sem.columns = tcprev_sem.columns.droplevel(0)
tcprev_sem.columns = ["aero", "diaSemana", "troca_prev_mean", "troca_prev_sum"]
tcreal_hora.reset_index(inplace=True)
tcreal_hora.columns = tcreal_hora.columns.droplevel(0)
tcreal_hora.columns = ["aero", "hora", "troca_real"]
tcreal_aero.reset_index(inplace=True)
tcreal_aero.columns = tcreal_aero.columns.droplevel(0)
tcreal_aero.columns = ["aero", "troca_real"]
tcreal_sem.reset_index(inplace=True)
tcreal_sem.columns = tcreal_sem.columns.droplevel(0)
tcreal_sem.columns = ["aero", "diaSemana", "troca_real"]


In [58]:
bimtra.columns

Index(['flightid', 'origem', 'destino', 'dt_dep', 'dt_arr', 'target', 'linha',
       'diaSemana', 'hora', 'aero_x', 'mean_x', 'sum_x', 'aero_y', 'mean_y',
       'sum_y', 'aero', 'mean', 'sum'],
      dtype='object')

In [60]:
bimtra = bimtra.merge(tcprev_hora, on=["aero", "hora"], how="left")
bimtra = bimtra.merge(tcprev_sem, on=["aero", "diaSemana"], how="left")
bimtra = bimtra.merge(tcprev_aero, on=["aero"], how="left")
bimtra = bimtra.merge(tcreal_hora, on=["aero", "hora"], how="left")
bimtra = bimtra.merge(tcreal_sem, on=["aero", "diaSemana"], how="left")
bimtra = bimtra.merge(tcreal_aero, on=["aero"], how="left")
bimtra

,flightid,origem,destino,dt_dep,dt_arr,target,linha,diaSemana,hora,aero_x,...,sum,troca_prev_mean_x,troca_prev_sum_x,troca_prev_mean_y,troca_prev_sum_y,troca_prev_mean,troca_prev_sum,troca_real_x,troca_real_y,troca_real
0,fcb2bf90345705318213ae1307c0f901,SBKP,SBRJ,2022-06-01 00:44:57,2022-06-01 01:26:00,2463,SBKPSBRJ,2,0,SBKP,...,1492,0.160494,52,0.293478,324,0.272119,2116,23.0,288,2060
1,c7c5c10716335b048f86d8c52fcba3f2,SBGR,SBRJ,2022-06-01 00:57:01,2022-06-01 01:32:53,2152,SBGRSBRJ,2,0,SBGR,...,1986,0.160494,52,0.293478,324,0.272119,2116,18.0,134,814
2,3162de7203a972f071d5a48e8f0f4828,SBBR,SBRJ,2022-06-01 00:26:52,2022-06-01 01:43:45,4613,SBBRSBRJ,2,0,SBBR,...,190,0.160494,52,0.293478,324,0.272119,2116,35.0,210,1153
3,8fcd243e9b9eebbe62ab145ce04b6ab5,SBKP,SBCF,2022-06-01 00:58:40,2022-06-01 01:41:41,2581,SBKPSBCF,2,0,SBKP,...,1492,0.160494,52,0.293478,324,0.272119,2116,23.0,288,2060
4,504a62621cd231d6ab67e674ce538cd3,SBCF,SBFL,2022-06-01 01:00:38,2022-06-01 02:33:05,5547,SBCFSBFL,2,1,SBCF,...,1170,0.160494,52,0.293478,324,0.272119,2116,25.0,109,912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262411,12d84d9dd7450b2cac83cdf041f58d67,SBGR,SBRJ,2023-04-15 21:41:11,2023-04-15 22:16:49,2138,SBGRSBRJ,5,21,SBGR,...,1986,0.259259,84,0.303442,335,0.272119,2116,85.0,96,814
262412,c1e1fcbfd1d1c67c01da384ad104c0df,SBBR,SBRJ,2023-04-15 21:07:10,2023-04-15 22:26:11,4741,SBBRSBRJ,5,21,SBBR,...,190,0.259259,84,0.303442,335,0.272119,2116,76.0,130,1153
262413,612af5b2b7cc48161bee87965785282b,SBFL,SBRJ,2023-04-15 21:30:27,2023-04-15 22:31:36,3669,SBFLSBRJ,5,21,SBFL,...,124,0.259259,84,0.303442,335,0.272119,2116,340.0,850,6151
262414,5efa9c4eb13e214f29a73a82cf7822e4,SBSV,SBRJ,2023-04-15 12:00:13,2023-04-15 14:00:05,7192,SBSVSBRJ,5,12,SBSV,...,72,0.351852,114,0.303442,335,0.272119,2116,590.0,1348,11140


In [61]:
train = bimtra.merge(resumo, right_on="linha_first", left_on="linha", how="left")

In [97]:
resumo

,linha_first,flightid_,dt_radar_<lambda>,dt_radar_<lambda>.1,dt_radar_<lambda>.2,dt_radar_<lambda>.3,lat_mean,lat_min,lat_max,lat_std,...,lon_max,lon_std,flightlevel_mean,flightlevel_min,flightlevel_max,flightlevel_std,speed_mean,speed_min,speed_max,speed_std
0,NaN,count,mean,min,max,std,mean,min,max,std,...,max,std,mean,min,max,std,mean,min,max,std
1,SBBRSBCF,2644,3210.6637670196674,0.0,8277.0,506.9815754584058,-0.3135663552228941,-0.4095989811,-0.2739212441,0.002530041610979571,...,-0.7026923893,0.0028508135030837728,223.23009635516672,1.0,410.0,15.414484379289918,365.208309469011,1.0,881.0,16.617049986812884
2,SBBRSBCT,959,5524.459854014599,4.0,8166.0,602.8147149739482,-0.36748848632055786,-0.4930038783,-0.1794724802,0.005855513262992156,...,-0.7082510707,0.0027255653742317646,281.071385902067,4.0,400.0,13.184672048521252,388.2540962465933,1.0,1555.0,13.50918093115723
3,SBBRSBFL,315,6614.263492063492,4.0,14041.0,824.4473638920381,-0.3867174708759314,-0.4862663929,-0.2765323694,0.006505166026804165,...,-0.7549999768,0.0017130580556180388,291.94466776671055,1.0,400.0,15.232888859932348,394.56650489239183,1.0,572.0,14.21473759918302
4,SBBRSBGL,639,4499.599374021909,362.0,7138.0,539.795847627226,-0.34150494094236883,-0.4146336416,-0.2324550936,0.004172015969809651,...,-0.6988433882,0.002983012702455903,259.66979013256446,1.0,607.0,13.252080132619195,393.2281877640942,1.0,552.0,15.93562582501301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,SBSVSBKP,1379,7368.569978245105,121.0,11225.0,761.98530512265,-0.31277198761030156,-0.4147787372,-0.0701902581,0.005460360215180889,...,-0.6646316611,0.004708078222442698,305.3446206649822,1.0,400.0,17.341259072228155,396.33274487402673,1.0,1286.0,16.681647582814314
134,SBSVSBPA,79,11771.683544303798,1683.0,13623.0,1455.2232848701617,-0.390669104767679,-0.525260494,-0.2250192926,0.010428194003739531,...,-0.6665442405,0.009512094269303203,324.43509942690906,1.0,400.0,8.228248797344037,392.19495205231044,34.0,490.0,8.362795726130809
135,SBSVSBRF,1491,3784.3105298457413,0.0,9182.0,526.7623314570509,-0.17761603549251634,-0.3543719597,-0.0650067621,0.002025703310561022,...,-0.6009507028,0.0013299560365280425,226.64509710011185,1.0,410.0,15.70704196991436,356.54213909988454,2.0,528.0,14.564714473730632
136,SBSVSBRJ,1630,6706.908588957055,843.0,10924.0,630.3015703620225,-0.3264385579605296,-0.5565484049,-0.1844531135,0.0038939621537456043,...,-0.6612747676,0.0027406425349698308,291.5118729723326,1.0,401.0,12.702016763699223,383.4791248139651,1.0,847.0,13.215119734561949


In [74]:
esperas_semana.to_csv("data/esperas_semana.csv", index=False)
esperas_hora.to_csv("data/esperas_hora.csv", index=False)

In [32]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

def calcular_esparsidade(dataframe):
    # Converter o DataFrame em uma matriz esparsa
    matriz_esparsa = csr_matrix(dataframe.values)

    # Contar o número de elementos não nulos
    elementos_nao_nulos = matriz_esparsa.nnz

    # Calcular o número total de elementos na matriz
    total_elementos = matriz_esparsa.shape[0] * matriz_esparsa.shape[1]

    # Calcular a esparsidade
    esparsidade = 1 - (elementos_nao_nulos / total_elementos)

    return esparsidade

# Exemplo de uso

esparsidade = calcular_esparsidade(esperas)

print("Esparsidade do DataFrame:", esparsidade)


Esparsidade do DataFrame: 0.9540991814698712


In [31]:
esperas

esperas    
                      mean sum
aero hora                     
SBBR 1654041600000     0.0   0
     1654045200000     0.0   0
     1654048800000     0.0   0
     1654052400000     0.0   0
     1654056000000     0.0   0
...                    ...  ..
SBSV 1681585200000     0.0   0
     1681588800000     0.0   0
     1681592400000     0.0   0
     1681596000000     0.0   0
     1681599600000     0.0   0

[91872 rows x 2 columns]

In [62]:
train["origem"], _ = pd.factorize(train["origem"])
train["destino"], _ = pd.factorize(train["destino"])


In [63]:
train["linha"], _ = pd.factorize(train["linha"])


In [64]:
del train["linha_first"]


In [65]:
del train["dt_dep"], train["dt_arr"], train["aero_x"]

In [66]:
del train["aero_y"]

In [67]:
del train["aero"]

In [70]:
train

,flightid,origem,destino,target,linha,diaSemana,hora,mean_x,sum_x,mean_y,...,lon_max,lon_std,flightlevel_mean,flightlevel_min,flightlevel_max,flightlevel_std,speed_mean,speed_min,speed_max,speed_std
0,fcb2bf90345705318213ae1307c0f901,0,0,2463,0,2,0,0.694444,225,0.221014,...,-0.7024397788,0.0033911938063553512,155.34488648758892,1.0,410.0,10.830585205248747,322.5658956783692,1.0,855.0,16.348628987169015
1,c7c5c10716335b048f86d8c52fcba3f2,1,0,2152,1,2,0,0.987654,320,0.254529,...,-0.6689091105,0.004944606649096218,132.86727827511172,1.0,471.0,9.681570140183254,305.9386740015981,1.0,576.0,16.3842888246202
2,3162de7203a972f071d5a48e8f0f4828,2,0,4613,2,2,0,0.000000,0,0.034420,...,-0.7188257205,0.0030770205896807823,261.70366305291907,1.0,411.0,14.824040870084831,384.16901657635884,1.0,1365.0,17.765934928708287
3,8fcd243e9b9eebbe62ab145ce04b6ab5,0,1,2581,3,2,0,0.694444,225,0.221014,...,-0.6918451014,0.002714306069815275,196.2949962157983,1.0,410.0,15.784746321116943,352.7619864507692,1.0,917.0,17.81165593918845
4,504a62621cd231d6ab67e674ce538cd3,3,2,5547,4,2,1,0.055556,18,0.173007,...,-0.7650227937,0.0015262837761787745,260.85033157991717,1.0,400.0,14.632191848997632,365.45367138614193,1.0,519.0,13.370615410380863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262411,12d84d9dd7450b2cac83cdf041f58d67,1,0,2138,1,5,21,0.123457,40,0.167572,...,-0.6689091105,0.004944606649096218,132.86727827511172,1.0,471.0,9.681570140183254,305.9386740015981,1.0,576.0,16.3842888246202
262412,c1e1fcbfd1d1c67c01da384ad104c0df,2,0,4741,2,5,21,0.074074,24,0.004529,...,-0.7188257205,0.0030770205896807823,261.70366305291907,1.0,411.0,14.824040870084831,384.16901657635884,1.0,1365.0,17.765934928708287
262413,612af5b2b7cc48161bee87965785282b,6,0,3669,37,5,21,0.033951,11,0.012681,...,-0.6872124596,0.003327307351236129,241.22151471637108,1.0,411.0,12.110501024534415,380.0117151986321,4.0,589.0,15.306431860431022
262414,5efa9c4eb13e214f29a73a82cf7822e4,10,0,7192,38,5,12,0.000000,0,0.003623,...,-0.6612747676,0.0027406425349698308,291.5118729723326,1.0,401.0,12.702016763699223,383.4791248139651,1.0,847.0,13.215119734561949


In [71]:

train.columns = [re.sub('[\[\]<>\'"\'.,]', '', col) for col in train.columns]
train.to_csv("data/train_cat_tc_esperas.csv", index=False)